In [1]:
%matplotlib inline
import numpy as np
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
from scipy.stats import poisson, kurtosis
import matplotlib.patches as mpatches
from functools import partial
import pickle
import os
import demixing as dm
from demixing import MLP, HiddenLayer

nneuron = 61
min_angle = -90
max_angle = 90
sprefs = np.linspace(min_angle, max_angle, nneuron)
eps = np.finfo(np.float64).eps
sigtc_sq = float(10**2)

In [ ]:
pkl_file = open('readout.pkl', 'rb')
posts, testsets = pickle.load(pkl_file)

In [31]:
posts = {}
testsets = {}
for s_i in range(31):
    #file_name = 'Minicon_Outputs/output_post_2/post_2_' + str(s_i) + '.pkl'
    file_name = 'output_post_3/post_3_' + str(s_i) + '.pkl'
    if os.path.isfile(file_name):
        pkl_file = open(file_name, 'rb')
        p, r, c, delta_s = pickle.load(pkl_file)
        posts[delta_s] = p
        testsets[delta_s] = r

In [28]:
nns = {}
for i in range(0, 200):
    #file_name = 'Minicon_Outputs/output_nn_tests_2/nn_tests_2_' + str(i) + '.pkl'
    file_name = 'output_nn_runs_2/nn_runs_2_' + str(i) + '.pkl'
    if os.path.isfile(file_name):
        pkl_file = open(file_name, 'rb')
        nn, nnx, valid_mse, _, _ = pickle.load(pkl_file)
        nns[i] = (nn, nnx, valid_mse)

In [ ]:
for i in range(1, len(nns) + 1):
    print i
    nn, nnx, valid_mse = nns[i]
    x = []
    y = []
    for s_i in range(31):
        x.append(dm.get_hu_responses(testsets[s_i], nn))
        y.append(np.array((1/posts[s_i]['var_s1'], 1/posts[s_i]['var_s2'])))
    y = np.concatenate(y, axis=1).T
    x = np.concatenate(x)
    inds = range(len(x))
    np.random.shuffle(inds)
    x_shuf = x[inds]
    y_shuf = y[inds]
    validset = x_shuf[0:2000], y_shuf[0:2000]
    trainset = x_shuf[2000:], y_shuf[2000:]
    
    hus, vpost = validset
    weights = np.linalg.lstsq(trainset[0], trainset[1])[0]
    lin_preds = np.dot(hus, weights)
    sum_preds = np.sum(hus, axis=1)
    kurt_preds = kurtosis(hus, axis=1)
    vp = np.concatenate((vpost.T[0], vpost.T[1]))
    lin_preds = np.concatenate((lin_preds.T[0], lin_preds.T[1]))
    sum_preds = np.concatenate((sum_preds, sum_preds))
    kurt_preds = np.concatenate((kurt_preds, kurt_preds))

    lin_corr = np.corrcoef(vp, lin_preds)
    sum_corr = np.corrcoef(vp, sum_preds)
    kurt_corr = np.corrcoef(vp, kurt_preds)

In [29]:
lin_corrs1 = np.zeros(200)
lin_corrs2 = np.zeros(200)

x2 = []
y = []
for s_i in range(31):
    x2.append(testsets[s_i])
    y.append(np.array((1/posts[s_i]['var_s1'], 1/posts[s_i]['var_s2'])))
y = np.concatenate(y, axis=1).T
x2 = np.concatenate(x2)
for i in range(0, 200):
    print i
    nn, nnx, valid_mse = nns[i]
    x1 = []
    for s_i in range(31):
        x1.append(dm.get_hu_responses(testsets[s_i], nn))      
    x1 = np.concatenate(x1)
    inds = range(len(x))
    np.random.shuffle(inds)
    x1_shuf = x1[inds]
    x2_shuf = x2[inds]
    y_shuf = y[inds]
    validset1 = x1_shuf[0:2000], y_shuf[0:2000]
    trainset1 = x1_shuf[2000:], y_shuf[2000:]
    validset2 = x2_shuf[0:2000], y_shuf[0:2000]
    trainset2 = x2_shuf[2000:], y_shuf[2000:]
    
    hus, vpost = validset1
    inputs, vpost = validset2
    weights1 = np.linalg.lstsq(trainset1[0], trainset1[1])[0]
    weights2 = np.linalg.lstsq(trainset2[0], trainset2[1])[0]
    lin_preds1 = np.dot(hus, weights1)
    lin_preds2 = np.dot(inputs, weights2)
    vp = np.concatenate((vpost.T[0], vpost.T[1]))
    lin_preds1 = np.concatenate((lin_preds1.T[0], lin_preds1.T[1]))
    lin_preds2 = np.concatenate((lin_preds2.T[0], lin_preds2.T[1]))

    lin_corrs1[i] = np.corrcoef(vp, lin_preds1)[0, 1]
    lin_corrs2[i] = np.corrcoef(vp, lin_preds2)[0, 1]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [39]:
np.mean(lin_corrs1 - lin_corrs2)

0.14483858247563405

In [38]:
lin_corrs2

array([ 0.67216138,  0.60160745,  0.6908572 ,  0.67458535,  0.44306168,
        0.67206579,  0.67044499,  0.66353416,  0.64587395,  0.62281688,
        0.63281637,  0.65690644,  0.64373559,  0.69099701,  0.6456014 ,
        0.65174826,  0.6903543 ,  0.67805748,  0.64676645,  0.68819446,
        0.63653439,  0.66032935,  0.69229795,  0.59427785,  0.68649242,
        0.62103585,  0.64054691,  0.66153746,  0.66842892,  0.70251546,
        0.4460853 ,  0.66124992,  0.66355753,  0.6396038 ,  0.65931336,
        0.68133755,  0.60830205,  0.69641675,  0.66584967,  0.62653081,
        0.64449835,  0.65093137,  0.65604053,  0.67742595,  0.65497473,
        0.67562996,  0.65790981,  0.6496472 ,  0.67478576,  0.66174989,
        0.63099614,  0.67665299,  0.66165093,  0.67075622,  0.64159055,
        0.54748562,  0.68626058,  0.66135856,  0.69218924,  0.68354712,
        0.70718138,  0.68941184,  0.67873863,  0.65470138,  0.59303042,
        0.66108682,  0.6239161 ,  0.50129741,  0.55654064,  0.44

In [41]:
lin_corrs1

array([ 0.83242001,  0.72315026,  0.82090456,  0.85136288,  0.5349494 ,
        0.79468366,  0.82869738,  0.84949768,  0.78386463,  0.7833485 ,
        0.69768106,  0.81355695,  0.7322944 ,  0.83716005,  0.77431337,
        0.79161608,  0.81753902,  0.85730901,  0.8012477 ,  0.86956482,
        0.80251526,  0.76951002,  0.84962828,  0.66265171,  0.85256264,
        0.78267495,  0.70700863,  0.85139602,  0.82846623,  0.80441147,
        0.5408431 ,  0.81386204,  0.8155565 ,  0.75746067,  0.84461864,
        0.84617551,  0.79492337,  0.86884891,  0.80703061,  0.73501817,
        0.83700891,  0.77400137,  0.84290623,  0.83790684,  0.80300168,
        0.81211911,  0.81368979,  0.76504155,  0.83337167,  0.83768287,
        0.73552042,  0.84611362,  0.8523619 ,  0.82258737,  0.79449986,
        0.72240352,  0.83725338,  0.82951635,  0.79576923,  0.82191297,
        0.82660613,  0.8362714 ,  0.83311743,  0.85054474,  0.69599263,
        0.85811433,  0.7398037 ,  0.52284967,  0.73720069,  0.54

In [38]:
in_corrs = np.zeros(201)
hu_corrs = np.zeros(201)
top2_corrs = np.zeros(201)
for i in range(201):
    file_name = 'output_post_readoutt2/post_readoutt2_2_' + str(i) + '.pkl'
    if os.path.isfile(file_name):
        pkl_file = open(file_name, 'rb')
        hu_corrs[i], top2_corrs[i], in_corrs[i], lin_preds_hus, lin_preds_top2, lin_preds_ins, weights_hus, weights_top2, weights_ins, sort_acts = pickle.load(pkl_file)

In [40]:
top2_corrs, in_corrs

(array([ 0.38824793,  0.49146123,  0.16345331,  0.31597994,  0.23999606,
         0.21926598,  0.08242616,  0.16157655,  0.31682509,  0.40101945,
         0.09091302,  0.29515735,  0.35969628,  0.38425458,  0.32704808,
         0.28624101,  0.17777761,  0.21171444,  0.11625638,  0.24964991,
         0.23363709,  0.03498055,  0.30664979,  0.28791036,  0.40760566,
         0.19060695,  0.33848829,  0.3496278 ,  0.34631485,  0.22690007,
         0.15305101,  0.37269119,  0.23928375,  0.34380986,  0.27681183,
         0.24087671,  0.35180481,  0.19563466,  0.14973164,  0.08427954,
         0.41083691,  0.3392105 ,  0.3139579 ,  0.19507934,  0.21460677,
         0.30186354,  0.25481872,  0.29196753,  0.28148589,  0.33803861,
         0.32585994,  0.19907264,  0.18462415,  0.3817493 ,  0.14199593,
         0.35092098,  0.36504595,  0.03035684,  0.27374714,  0.34543487,
         0.42934289,  0.24686766,  0.13999865,  0.27071676,  0.27370018,
         0.21497778,  0.3789258 ,  0.44333951,  0.1

In [23]:
nns = {}
for i in range(201):
    file_name = 'output_nn_runs_cov/nn_runs_2_' + str(i) + '.pkl'
    if os.path.isfile(file_name):
        pkl_file = open(file_name, 'rb')
        nn, nnx, valid_mse, _, _ = pickle.load(pkl_file)
        nns[i] = (nn, nnx, valid_mse)

In [32]:
lin_corrs1 = np.zeros(200)
lin_corrs2 = np.zeros(200)

x2 = []
y = []
for s_i in range(31):
    x2.append(testsets[s_i])
    y.append(np.array((1/posts[s_i]['var_s1'], 1/posts[s_i]['var_s2'])))
y = np.concatenate(y, axis=1).T
x2 = np.concatenate(x2)
for i in range(201):
    print i
    nn, nnx, valid_mse = nns[i]
    x1 = []
    for s_i in range(31):
        x1.append(dm.get_hu_responses(testsets[s_i], nn))      
    x1 = np.concatenate(x1)
    inds = range(len(x1))
    np.random.shuffle(inds)
    x1_shuf = x1[inds]
    x2_shuf = x2[inds]
    y_shuf = y[inds]
    validset1 = x1_shuf[0:2000], y_shuf[0:2000]
    trainset1 = x1_shuf[2000:], y_shuf[2000:]
    validset2 = x2_shuf[0:2000], y_shuf[0:2000]
    trainset2 = x2_shuf[2000:], y_shuf[2000:]
    
    hus, vpost = validset1
    inputs, vpost = validset2
    weights1 = np.linalg.lstsq(trainset1[0], trainset1[1])[0]
    weights2 = np.linalg.lstsq(trainset2[0], trainset2[1])[0]
    lin_preds1 = np.dot(hus, weights1)
    lin_preds2 = np.dot(inputs, weights2)
    vp = np.concatenate((vpost.T[0], vpost.T[1]))
    lin_preds1 = np.concatenate((lin_preds1.T[0], lin_preds1.T[1]))
    lin_preds2 = np.concatenate((lin_preds2.T[0], lin_preds2.T[1]))
    sum_preds = np.sum(hus, axis=1)
    kurt_preds = kurtosis(hus, axis=1)
    sum_preds = np.concatenate((sum_preds, sum_preds))
    kurt_preds = np.concatenate((kurt_preds, kurt_preds))

    sum_corr = np.corrcoef(vp, sum_preds)
    kurt_corr = np.corrcoef(vp, kurt_preds)
    lin_corrs1[i] = np.corrcoef(vp, lin_preds1)[0, 1]
    lin_corrs2[i] = np.corrcoef(vp, lin_preds2)[0, 1]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62


LinAlgError: SVD did not converge in Linear Least Squares

In [41]:
sum_corr

array([[ 1.        ,  0.06988951],
       [ 0.06988951,  1.        ]])

In [37]:
weights1

array([[ -5.30542786e-02,   1.58300374e-02],
       [ -2.46917685e-03,   2.78910569e-02],
       [  3.31833549e-02,   3.95405797e-03],
       [  1.92035109e-01,  -2.20592956e-02],
       [ -5.41356689e-02,   1.45408297e+00],
       [  3.55643253e-01,   6.64316530e-03],
       [ -1.24587622e+00,   1.07341108e-02],
       [ -3.06208394e-04,  -2.07341218e-03],
       [  5.25715962e-03,   4.26600225e-02],
       [ -7.15703757e-02,  -5.70746612e-02],
       [  4.62333989e-03,  -1.13843572e-01],
       [  2.23866676e-02,  -1.24884140e-02],
       [ -6.12754144e-02,   2.27170949e-02],
       [  2.61498364e-02,   1.67238804e-02],
       [ -3.89719167e-02,  -1.23261308e-01],
       [ -4.96895581e-02,   8.08292133e-01],
       [  1.48532600e-02,  -2.73340635e-02],
       [ -1.39452264e-01,   1.07757940e-01],
       [  2.31934784e-02,  -4.62203297e-02],
       [  1.72943068e-01,  -2.09367523e-02],
       [ -4.65059693e-02,  -3.49948465e-02],
       [ -7.03173590e-02,   2.91514787e-01],
       [ -